In [1]:
import cv2
import numpy as np
import cupy as cp
import time
import imutils
from joblib import load
from dataset_io import *
import timeit
from tqdm import tqdm
import psutil
from datetime import datetime
import cProfile

In [2]:
def pyramid(image, split_channels, initial_scale, final_scale, scale):
    original_w=image.shape[1]
    curr_scale=initial_scale
    while curr_scale>final_scale/scale:
        w = int(original_w * curr_scale)
        yield imutils.resize(image, width=w), [imutils.resize(ch, width=w) for ch in split_channels],curr_scale
        curr_scale/=scale
        
def sliding_window(image, stepSize, windowSize):
    for y in range(0, image.shape[0], stepSize):
        for x in range(0, image.shape[1], stepSize):
            yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])

In [3]:
def area(rect):
    ((x,y),(x2,y2))=rect
    return (x2-x)*(y2-y)

def calc_overlapping(a,b,method):
    dx = min(a[1][0], b[1][0]) - max(a[0][0], b[0][0])
    dy = min(a[1][1], b[1][1]) - max(a[0][1], b[0][1])
    basic_area=method([area(a), area(b)])
    overlap_area=dx*dy
    is_overlapping=((dx>=0) and (dy>=0))
    overlapping_ratio=overlap_area/basic_area
    
    return overlapping_ratio if is_overlapping else 0

def scale_many(vals, scale):
    return [int(val/scale) for val in vals]

## Load classifier

In [ ]:
os.chdir(PROJECT_PATH)
clf = load('rand_forest_clf.joblib') 
clf.set_params(n_jobs=-1)

## Set params

In [ ]:
(winW, winH) = (120,24)
channels_no=8
DS_min_winH_m=41
DS_max_winH_m=287
initial_scale=winH/DS_min_winH_m
final_scale=winH/DS_max_winH_m
RESIZING_SCALE=1.15
MAX_PRED_OVERLAPPING=0.5
MIN_PRED=0.8
WHITE=(255, 255, 255)
DRAW_BORDER=2

In [4]:
def show(img, x, y, winW, winH, positives=[]):
    clone = img.copy()
    cv2.rectangle(clone, (x, y), (x + winW, y + winH), WHITE, DRAW_BORDER)
    for p in positives:
        x,y=p
        cv2.rectangle(clone, (x, y), (x + winW, y + winH), WHITE, DRAW_BORDER)
    cv2.imshow("Window", clone)
    cv2.waitKey(1)

def pred_overlapping(row, scaled_rect):
    x,y,h,w = scaled_rect
    a = ((x,y),(x+w,y+h))
    [scene_name, filename, scale, (rx,ry)] = row
    [rx,ry,rw,rh] = scale_many([rx,ry,winW,winH], scale)
    b = ((rx,ry),(rx+rw,ry+rh))

    return calc_overlapping(a,b,np.min)
    
def image_predict(X,IDX,rects_count):
    global glob_RES
    if len(X)!=0 and rects_count>0:
        pred=np.array([p[1] for p in clf.predict_proba(X)])
        greater_than_09=pred > MIN_PRED
        pred=pred[greater_than_09]
        IDX=IDX[greater_than_09]
        max_pred_ids=[]
        max_preds=[]
        max_pred_IDXs=[]
        for i in range(rects_count):
            if len(pred) >0:
                max_pred_id=np.argmax(pred)
                max_pred=pred[max_pred_id]
                max_preds.append(max_pred)
                max_pred_IDX=IDX[max_pred_id]
                max_pred_IDXs.append(max_pred_IDX)

                [scene_name, filename, scale, (x,y)]=max_pred_IDX
                scaled_rect=scale_many([x,y,winW,winH], scale)

                not_within_bools = np.array([pred_overlapping(row, scaled_rect) < MAX_PRED_OVERLAPPING for row in IDX])
                pred=pred[not_within_bools]
                IDX=IDX[not_within_bools]
            else:
                print("No more predictions with prob > 0.8")
        if len(max_preds)>0:
            max_preds=np.array([[e] for e in max_preds])
            res=np.append(max_pred_IDXs, max_preds, 1)
            glob_RES=np.vstack([glob_RES,res])
    
def predict(filename, scene_name, label_resolution, rects):
    X=[]
    IDX=[]
    
    image=imread_resized(scene_name, filename, label_resolution)
    split_channels=read_split_channels(scene_name, filename)
    for resized_img, resized_split_ch, scale in pyramid(image, split_channels, initial_scale, final_scale, scale=RESIZING_SCALE):
        resized_ch=np.dstack(resized_split_ch)
        for (x, y, window) in sliding_window(resized_img, stepSize=4, windowSize=(winW, winH)):
            # if the window does not meet our desired window size, ignore it
            if window.shape[0] != winH or window.shape[1] != winW:
                continue
            channels_window=resized_ch[y:y + winH, x:x + winW]
            X.append(channels_window)
            row=[scene_name, filename, scale, (x,y)]
            IDX.append(row)
    X=np.reshape(X,(len(X),winW*winH*channels_no)) #cupy does not help
    image_predict(X,np.array(IDX), len(rects))

In [5]:
glob_RES=np.empty((0,5), object)

glob_DATETIME = datetime.now().strftime("%d-%m-%Y_%H:%M:%S")

def predict_all():
    [predict(*row) for row in tqdm([row for row in walk_dataset()])]# if row[1]=="warehouse_1"])]# and row[0]=="r_1_0.jpg"])]

In [6]:
cProfile.runctx(
            'predict_all()',
            globals(),
            locals(),
            'myProfilingFile.pstats'
        )
os.chdir(PROJECT_PATH)
!python3 /usr/local/lib/python3.6/dist-packages/gprof2dot.py -f pstats myProfilingFile.pstats | dot -Tpng -o image_output.png

 11%|█         | 225/2092 [06:25<59:43,  1.92s/it]  

No more predictions with prob > 0.8


 11%|█         | 235/2092 [06:44<58:12,  1.88s/it]

No more predictions with prob > 0.8


 12%|█▏        | 243/2092 [06:59<59:36,  1.93s/it]  

...

 96%|█████████▋| 2014/2092 [57:41<01:57,  1.51s/it]

No more predictions with prob > 0.8


 96%|█████████▋| 2016/2092 [57:44<01:54,  1.51s/it]

No more predictions with prob > 0.8


100%|██████████| 2092/2092 [1:00:03<00:00,  1.72s/it]


In [11]:
def calc_overlappings(rects, pred_rects):
    global glob_overlappings    
    overlappings=[(np.max([calc_overlapping(pred_rect,rect, np.max) for pred_rect in pred_rects]) if len(pred_rects) > 0 else 0) for rect in rects]
    glob_overlappings.extend(overlappings)

def draw_predicted_rectangles(filename, scene_name, label_resolution, rects):
    global glob_RES
    
    img=imread_resized(scene_name, filename, label_resolution)
    img_rows=glob_RES[np.where((glob_RES[:,0] == scene_name) * (glob_RES[:,1] == filename))]
    
    pred_rects=[]
    rects=[add_margin(correct_rect_ratio(rect)) for rect in rects]
    for max_row in img_rows:
        [scene_name, filename, scale, (x,y), pred]=max_row
        [x,y,winW_s,winH_s]=scale_many([x,y,winW,winH], scale)
        cv2.rectangle(img, (x,y), (x + winW_s, y + winH_s), (0, 255, 0), 2)
        pred_rects.append(((x,y),(x+winW_s, y+winH_s)))
    for rect in rects: 
        ((x,y),(x2,y2))=rect
        cv2.rectangle(img, (x,y), (x2,y2), (255, 0, 0), 2)
    
    save_image(img, scene_name, filename, "predicted_labels")
    calc_overlappings(rects, pred_rects)

glob_overlappings=[]
[draw_predicted_rectangles(*row) for row in tqdm(walk_dataset())]# if row[1]=="warehouse_1"]# and row[0]=="r_1_0.jpg"]
print("Mean overlapping: ",np.mean(glob_overlappings))


 24%|██▍       | 498/2092 [00:27<00:16, 95.39it/s]


 67%|██████▋   | 1400/2092 [00:49<01:00, 11.39it/s]


 81%|████████  | 1696/2092 [01:16<00:38, 10.40it/s]


 95%|█████████▌| 1992/2092 [01:42<00:08, 11.87it/s]


100%|██████████| 2092/2092 [01:45<00:00, 19.86it/s]

Mean overlapping:  0.6239104033578537


In [ ]:
#min_pred=0.8 : overlap = 0.623